# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770499392190                   -0.52    9.0    187ms
  2   -2.771695400289       -2.92       -1.32    1.0   95.3ms
  3   -2.771714434990       -4.72       -2.47    1.0    102ms
  4   -2.771714647772       -6.67       -3.18    1.0    101ms
  5   -2.771714714325       -7.18       -4.02    2.0    120ms
  6   -2.771714715092       -9.11       -4.53    1.0    107ms
  7   -2.771714715247       -9.81       -5.57    1.0    108ms
  8   -2.771714715249      -11.57       -5.82    2.0    125ms
  9   -2.771714715250      -12.57       -6.98    1.0    113ms
 10   -2.771714715250      -14.35       -7.42    2.0    129ms
 11   -2.771714715250   +  -15.05       -8.39    1.0    118ms


-0.00013457348631932905

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770494613418                   -0.53    8.0    156ms
  2   -2.771775783571       -2.89       -1.32    1.0    108ms
  3   -2.771801705679       -4.59       -2.51    1.0    134ms
  4   -2.771802025554       -6.50       -3.31    1.0    131ms
  5   -2.771802073142       -7.32       -3.93    2.0    155ms
  6   -2.771802074401       -8.90       -4.72    1.0    145ms
  7   -2.771802074472      -10.15       -5.19    2.0    119ms
  8   -2.771802074476      -11.39       -6.40    1.0    106ms
  9   -2.771802074476      -13.13       -6.60    2.0    127ms
 10   -2.771802074476      -14.12       -7.30    1.0    108ms
 11   -2.771802074476   +  -14.31       -7.98    2.0    122ms
 12   -2.771802074476      -14.24       -8.25    1.0    110ms


0.01761222037224919

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457348631932905
Displaced dipole:  0.01761222037224919
Polarizability :   1.774679385856852


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, -δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0   5.24s  Non-interacting
   1        0       1       -0.60     10.0   6.57s  
   2        0       2       -2.42      8.0   455ms  
   3        0       3       -3.54      5.0   105ms  
   4        0       4       -5.33      4.0   101ms  
   5        0       5       -7.37      1.0  86.3ms  
   6        0       6      -10.09      1.0  87.7ms  
   7        1       1       -7.37     13.0   332ms  Restart
   8        1       2       -8.93      1.0   104ms  
                                      13.0   578ms  Final orbitals


(δψ = Matrix{ComplexF64}[[0.0012502934703486088 - 0.0030495382120732653im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.2852746413695871 - 0.119768840038661im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; -0.03793257909298792 - 0.015731061371274245im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.07500542972721164 + 0.030416729978593203im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.620692641965172e-7 -2.502013459747132e-7 … -4.8806392811344464e-8 -2.502014029904075e-7; -6.293613342864505e-7 -4.793033495422785e-7 … -1.2077829659286908e-7 -4.793033769004638e-7; … ; 1.3626248522079236e-7 1.1356217451756004e-7 … 4.711977432037362e-8 1.1356216682307974e-7; 1.3305466723923665e-7 1.363263425029296e-7 … 5.2006785300760453e-8 1.363262940728241e-7;;; -2.502013924440091e-7 -1.7404896158854688e-7 … -3.5564906901439426e-8 -1.7404900332144277e-7; -4.793033809916441e-7 -3.6797296320050697e-7 … -1.1202880569309052e-7 -3.679729853157594e-7; … ; 1.1356217154004521e-7 1.1211084779427494e-7 … 1.0980666403162982e-7 1.121

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125429562696
Interacting polarizability:     1.7736548717454692


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 8.31e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 7.04e-11
┌ Info: GMRES linsolve converged at iteration 2, step 2:
│ * norm of residual = 3.60e-12
└ * number of operations = 11
Non-interacting polarizability: 1.9257125429561663
Interacting polarizability:     1.7736548665388858


We obtain the identical result to above.